In [1]:
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from pydantic import BaseModel, Field
from textwrap import dedent

from ragas import EvaluationDataset, RunConfig, evaluate, SingleTurnSample
from ragas.metrics import (LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from operator import itemgetter
import pandas as pd
import random
import os
import re

from loader import load_process_split_doc, load_process_split_doc_law

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [ ]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

folders = ["data/tax_law1", "data/tax_law2", "data/tax_law3", "data/tax_law4", "data/tax_law5", "data/tax_law6"]

# vector store에 세법 저장.
for folder in folders:
    law_files = get_file_names(folder)
    for file in law_files:
        documents=load_process_split_doc_law(file, directory_path=folder)
        vector_store.add_documents(documents)

# tax_etc 파일 전처리리
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# 참고자료 vector store에 저장.
for doc in doc_options:
    vector_store.add_documents(load_process_split_doc(**doc))

vector_store.persist()

all_documents = []

# tax law 를 all_documents에 저장.
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file, directory_path="data/tax_law"))

# 참고자료 를 all_documents에 저장.
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))

C:\Users\Playdata\AppData\Local\Temp\ipykernel_3392\2552248468.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_3392\2552248468.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [3]:
# Retreiver 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 
    문서에 없는 내용일 경우 모른다고 대답해주세요.
    
    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온 후 요약 정리하여 이해하게 설명한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계한다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."
    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [4]:
def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """
    list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환
    """
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """
    list[Document]에서 page_content 값들만 추출한 list를 반환
    """
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough()
    |{"context":retriever, "question":RunnablePassthrough()}
    |
    {"source_context" : itemgetter("context") | RunnableLambda(str_from_documents), "llm_answer":{"context": RunnableLambda(format_docs), "question":itemgetter("question")}
        |prompt_template
        |model
        |parser
    }
)

In [5]:
class EvalDatasetSchema(BaseModel):
    user_input:str = Field(..., description="질문(Question)")
    retrieved_contexts:list[str] = Field(..., description="LLM이 답변할 때 참조할 context")
    reference: str = Field(..., description="정답(ground truth)")

jsonparser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

qa_prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개의 질문을 생성하세요. 

        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [context]에 없는 내용을 가지고 질문-답변을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 답변을 만들고 그 내용이 [context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어 주십시오.
                    
        출력 형식: {format_instructions}

        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":jsonparser.get_format_instructions()}
)

# 데이터셋 생성 체인 구성
model = ChatOpenAI(model="gpt-4o")
dataset_generator_chain = qa_prompt_template | model | jsonparser

In [6]:
# 평가 데이터로 사용할 5개의 context 추출
total_samples = 5

idx_list = list(range(len(all_documents)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = all_documents[idx].page_content
    if len(context) > 100:
        eval_context_list.append(context)

In [7]:
# 전체 context sample들로 qa dataset을 생성
eval_data_list = []
num_questions = 2
for context in eval_context_list:
    _eval_data_list = dataset_generator_chain.invoke(
        {"context":context, "num_questions":num_questions}
    )
    for eval_data in _eval_data_list:
        eval_data['retrieved_contexts'] = [context]
    
    eval_data_list.extend(_eval_data_list)

eval_df = pd.DataFrame(eval_data_list)

context_list = []
response_list = []

for user_input in eval_df['user_input']:
    res = rag_chain.invoke(user_input)
    context_list.append(res['source_context'])
    response_list.append(res['llm_answer'])

eval_df["retrieved_contexts"] = context_list
eval_df["response"] = response_list

eval_dataset = EvaluationDataset.from_pandas(eval_df)

In [11]:
load_dotenv()
model = ChatOpenAI(model= "gpt-4o")
eval_llm = LangchainLLMWrapper(model)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


In [ ]:
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)
result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Exception raised in Job[4]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[0]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[18]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[22]: TimeoutError()


KeyError: 0

In [18]:
# metrics1 = [LLMContextRecall(llm=eval_llm)]
# metrics2 = [LLMContextPrecisionWithReference(llm=eval_llm)]
# metrics3 = [Faithfulness(llm=eval_llm)]
# metrics4 = [AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)]

In [19]:
# result_recall = evaluate(dataset=eval_dataset, metrics=metrics1)
# result_recall.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_recall
0,국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때 어떤 요청을 할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,"질문에 대한 답변: 국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때, 통계...","국세청장은 통계자료의 사용 목적, 사용 방법 등을 제한하거나 통계자료의 안전성 확보...",1.0
1,정부출연연구기관의 장은 어떤 방식으로 통계자료를 요청할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,정부출연연구기관의 장은 통계자료를 요청할 때 다음과 같은 사항을 적은 문서를 국세청...,"정부출연연구기관의 장은 통계자료의 명칭, 사용 목적, 내용과 범위, 제공방법 등을 ...",1.0
2,제9조에 따르면 지방국세청장 또는 세무서장은 압수물건을 공매할 때 어떤 정보를 공고...,[① 관할 세무서장은 법 제67조에 따라 압류재산을 수의계약으로 매각하려는 경우 추...,죄송합니다. 해당 질문에 대한 답변을 제공하기 위해서는 관련 법령의 전문을 확인해야...,"물건의 품명, 수량, 공매 사유, 공매 장소와 그 일시, 그 밖에 필요한 사항을 공...",1.0
3,세무공무원이 압수물건을 매수할 수 있나요?,[제2항에 따라 압류를 한 후 압류에 따라 징수하려는 국세를 확정한 경우 압류한 재...,세무공무원은 압수한 물건을 매수할 수 없습니다. 세무공무원이 직무를 수행하면서 압수...,세무공무원은 압수물건을 직접 또는 간접으로 매수할 수 없다.,1.0
4,장려세제과장은 어떤 업무를 담당합니까?,[기획 및 조사대상자 선정기준 마련 2. 주식 및 파생상품 관련 정보자료ㆍ과세자료의...,장려세제과장은 다음과 같은 업무를 담당합니다:\n\n1. 근로장려세제 및 자녀장려세...,"장려세제과장은 근로장려세제 및 자녀장려세제의 홍보와 상담, 신청내용의 심사와 지급결...",1.0
5,국세공무원교육원 교육운영과장은 어떤 업무를 담당합니까?,[보한다. ③장려세제과장은 다음 사항을 분장한다. 1. 근로장려세제 및 자녀장려세제...,질문에 대한 답변: 국세공무원교육원 교육운영과장은 교육과 관련된 다양한 업무를 담당...,"교육운영과장은 교육계획의 수립, 교과시간의 운영, 강사관리, 교육생의 선발 및 학적...",1.0
6,황정연의 소득금액은 얼마인가?,[Ⅱ. 연금소득 연말정산 351 09 연금소득 원천징수영수증(연말정산용) 작성김일일...,질문이 명확하지 않습니다. 황정연이라는 이름과 관련된 구체적인 상황이나 정보를 제공...,"황정연의 소득금액은 4,700,000원입니다.",0.0
7,이태영은 어떤 공제 혜택을 받는가?,"[핵심 개정 세법 104대토보상 과세특례 적용요건 보완 (조특법 §77의2②, 조특...",죄송합니다. 이태영이라는 이름으로 특정 공제 혜택을 제공하는 세법 정보는 제가 보유...,"이태영은 국세청에서 1,500,000원의 소득공제 혜택을 받습니다.",0.0
8,봉사료가 유흥음식요금에 포함되지 않는 경우는?,"[개별소비세법 시행령 기획재정부 (환경에너지세제과) 044-215-4331, 433...",봉사료가 유흥음식요금에 포함되지 않는 경우는 다음과 같습니다:\n\n유흥음식요금에 ...,봉사료가 해당 종업원에게 지급된 사실이 확인되는 경우에는 유흥음식요금에 포함되지 않는다.,1.0
9,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은?,[143 ❚상세본 07 기타 법률수소제조용 석유가스(LPG) 부탄에 대한 환급특례 ...,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 다음과 같습니다:\n\n1....,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 킬로그램당 176.4원 또는...,1.0


In [ ]:
# result_precision = evaluate(dataset=eval_dataset, metrics=metrics2)
# result_precision.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,llm_context_precision_with_reference
0,국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때 어떤 요청을 할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,"질문에 대한 답변: 국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때, 통계...","국세청장은 통계자료의 사용 목적, 사용 방법 등을 제한하거나 통계자료의 안전성 확보...",1.000000
1,정부출연연구기관의 장은 어떤 방식으로 통계자료를 요청할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,정부출연연구기관의 장은 통계자료를 요청할 때 다음과 같은 사항을 적은 문서를 국세청...,"정부출연연구기관의 장은 통계자료의 명칭, 사용 목적, 내용과 범위, 제공방법 등을 ...",0.887500
2,제9조에 따르면 지방국세청장 또는 세무서장은 압수물건을 공매할 때 어떤 정보를 공고...,[① 관할 세무서장은 법 제67조에 따라 압류재산을 수의계약으로 매각하려는 경우 추...,죄송합니다. 해당 질문에 대한 답변을 제공하기 위해서는 관련 법령의 전문을 확인해야...,"물건의 품명, 수량, 공매 사유, 공매 장소와 그 일시, 그 밖에 필요한 사항을 공...",0.950000
3,세무공무원이 압수물건을 매수할 수 있나요?,[제2항에 따라 압류를 한 후 압류에 따라 징수하려는 국세를 확정한 경우 압류한 재...,세무공무원은 압수한 물건을 매수할 수 없습니다. 세무공무원이 직무를 수행하면서 압수...,세무공무원은 압수물건을 직접 또는 간접으로 매수할 수 없다.,0.950000
4,장려세제과장은 어떤 업무를 담당합니까?,[기획 및 조사대상자 선정기준 마련 2. 주식 및 파생상품 관련 정보자료ㆍ과세자료의...,장려세제과장은 다음과 같은 업무를 담당합니다:\n\n1. 근로장려세제 및 자녀장려세...,"장려세제과장은 근로장려세제 및 자녀장려세제의 홍보와 상담, 신청내용의 심사와 지급결...",0.866667
5,국세공무원교육원 교육운영과장은 어떤 업무를 담당합니까?,[보한다. ③장려세제과장은 다음 사항을 분장한다. 1. 근로장려세제 및 자녀장려세제...,질문에 대한 답변: 국세공무원교육원 교육운영과장은 교육과 관련된 다양한 업무를 담당...,"교육운영과장은 교육계획의 수립, 교과시간의 운영, 강사관리, 교육생의 선발 및 학적...",1.000000
6,황정연의 소득금액은 얼마인가?,[Ⅱ. 연금소득 연말정산 351 09 연금소득 원천징수영수증(연말정산용) 작성김일일...,질문이 명확하지 않습니다. 황정연이라는 이름과 관련된 구체적인 상황이나 정보를 제공...,"황정연의 소득금액은 4,700,000원입니다.",0.000000
7,이태영은 어떤 공제 혜택을 받는가?,"[핵심 개정 세법 104대토보상 과세특례 적용요건 보완 (조특법 §77의2②, 조특...",죄송합니다. 이태영이라는 이름으로 특정 공제 혜택을 제공하는 세법 정보는 제가 보유...,"이태영은 국세청에서 1,500,000원의 소득공제 혜택을 받습니다.",0.000000
8,봉사료가 유흥음식요금에 포함되지 않는 경우는?,"[개별소비세법 시행령 기획재정부 (환경에너지세제과) 044-215-4331, 433...",봉사료가 유흥음식요금에 포함되지 않는 경우는 다음과 같습니다:\n\n유흥음식요금에 ...,봉사료가 해당 종업원에게 지급된 사실이 확인되는 경우에는 유흥음식요금에 포함되지 않는다.,0.750000
9,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은?,[143 ❚상세본 07 기타 법률수소제조용 석유가스(LPG) 부탄에 대한 환급특례 ...,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 다음과 같습니다:\n\n1....,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 킬로그램당 176.4원 또는...,1.000000


In [ ]:
# result_faithfulness = evaluate(dataset=eval_dataset, metrics=metrics3)
# result_faithfulness.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness
0,국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때 어떤 요청을 할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,"질문에 대한 답변: 국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때, 통계...","국세청장은 통계자료의 사용 목적, 사용 방법 등을 제한하거나 통계자료의 안전성 확보...",0.500000
1,정부출연연구기관의 장은 어떤 방식으로 통계자료를 요청할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,정부출연연구기관의 장은 통계자료를 요청할 때 다음과 같은 사항을 적은 문서를 국세청...,"정부출연연구기관의 장은 통계자료의 명칭, 사용 목적, 내용과 범위, 제공방법 등을 ...",0.800000
2,제9조에 따르면 지방국세청장 또는 세무서장은 압수물건을 공매할 때 어떤 정보를 공고...,[① 관할 세무서장은 법 제67조에 따라 압류재산을 수의계약으로 매각하려는 경우 추...,죄송합니다. 해당 질문에 대한 답변을 제공하기 위해서는 관련 법령의 전문을 확인해야...,"물건의 품명, 수량, 공매 사유, 공매 장소와 그 일시, 그 밖에 필요한 사항을 공...",0.500000
3,세무공무원이 압수물건을 매수할 수 있나요?,[제2항에 따라 압류를 한 후 압류에 따라 징수하려는 국세를 확정한 경우 압류한 재...,세무공무원은 압수한 물건을 매수할 수 없습니다. 세무공무원이 직무를 수행하면서 압수...,세무공무원은 압수물건을 직접 또는 간접으로 매수할 수 없다.,0.375000
4,장려세제과장은 어떤 업무를 담당합니까?,[기획 및 조사대상자 선정기준 마련 2. 주식 및 파생상품 관련 정보자료ㆍ과세자료의...,장려세제과장은 다음과 같은 업무를 담당합니다:\n\n1. 근로장려세제 및 자녀장려세...,"장려세제과장은 근로장려세제 및 자녀장려세제의 홍보와 상담, 신청내용의 심사와 지급결...",0.666667
5,국세공무원교육원 교육운영과장은 어떤 업무를 담당합니까?,[보한다. ③장려세제과장은 다음 사항을 분장한다. 1. 근로장려세제 및 자녀장려세제...,질문에 대한 답변: 국세공무원교육원 교육운영과장은 교육과 관련된 다양한 업무를 담당...,"교육운영과장은 교육계획의 수립, 교과시간의 운영, 강사관리, 교육생의 선발 및 학적...",0.400000
6,황정연의 소득금액은 얼마인가?,[Ⅱ. 연금소득 연말정산 351 09 연금소득 원천징수영수증(연말정산용) 작성김일일...,질문이 명확하지 않습니다. 황정연이라는 이름과 관련된 구체적인 상황이나 정보를 제공...,"황정연의 소득금액은 4,700,000원입니다.",0.400000
7,이태영은 어떤 공제 혜택을 받는가?,"[핵심 개정 세법 104대토보상 과세특례 적용요건 보완 (조특법 §77의2②, 조특...",죄송합니다. 이태영이라는 이름으로 특정 공제 혜택을 제공하는 세법 정보는 제가 보유...,"이태영은 국세청에서 1,500,000원의 소득공제 혜택을 받습니다.",0.250000
8,봉사료가 유흥음식요금에 포함되지 않는 경우는?,"[개별소비세법 시행령 기획재정부 (환경에너지세제과) 044-215-4331, 433...",봉사료가 유흥음식요금에 포함되지 않는 경우는 다음과 같습니다:\n\n유흥음식요금에 ...,봉사료가 해당 종업원에게 지급된 사실이 확인되는 경우에는 유흥음식요금에 포함되지 않는다.,0.800000
9,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은?,[143 ❚상세본 07 기타 법률수소제조용 석유가스(LPG) 부탄에 대한 환급특례 ...,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 다음과 같습니다:\n\n1....,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 킬로그램당 176.4원 또는...,0.833333


In [22]:
# result_relevancy = evaluate(dataset=eval_dataset, metrics=metrics4)
# result_relevancy.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,answer_relevancy
0,국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때 어떤 요청을 할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,"질문에 대한 답변: 국세청장은 정부출연연구기관의 장에게 통계자료를 제공할 때, 통계...","국세청장은 통계자료의 사용 목적, 사용 방법 등을 제한하거나 통계자료의 안전성 확보...",0.0
1,정부출연연구기관의 장은 어떤 방식으로 통계자료를 요청할 수 있나요?,[생산 할 수 없는 것인 경우에는 그 사유를 첨부하여 소관 상임위원회에 통보하여야 ...,정부출연연구기관의 장은 통계자료를 요청할 때 다음과 같은 사항을 적은 문서를 국세청...,"정부출연연구기관의 장은 통계자료의 명칭, 사용 목적, 내용과 범위, 제공방법 등을 ...",0.0
2,제9조에 따르면 지방국세청장 또는 세무서장은 압수물건을 공매할 때 어떤 정보를 공고...,[① 관할 세무서장은 법 제67조에 따라 압류재산을 수의계약으로 매각하려는 경우 추...,죄송합니다. 해당 질문에 대한 답변을 제공하기 위해서는 관련 법령의 전문을 확인해야...,"물건의 품명, 수량, 공매 사유, 공매 장소와 그 일시, 그 밖에 필요한 사항을 공...",0.0
3,세무공무원이 압수물건을 매수할 수 있나요?,[제2항에 따라 압류를 한 후 압류에 따라 징수하려는 국세를 확정한 경우 압류한 재...,세무공무원은 압수한 물건을 매수할 수 없습니다. 세무공무원이 직무를 수행하면서 압수...,세무공무원은 압수물건을 직접 또는 간접으로 매수할 수 없다.,0.0
4,장려세제과장은 어떤 업무를 담당합니까?,[기획 및 조사대상자 선정기준 마련 2. 주식 및 파생상품 관련 정보자료ㆍ과세자료의...,장려세제과장은 다음과 같은 업무를 담당합니다:\n\n1. 근로장려세제 및 자녀장려세...,"장려세제과장은 근로장려세제 및 자녀장려세제의 홍보와 상담, 신청내용의 심사와 지급결...",0.0
5,국세공무원교육원 교육운영과장은 어떤 업무를 담당합니까?,[보한다. ③장려세제과장은 다음 사항을 분장한다. 1. 근로장려세제 및 자녀장려세제...,질문에 대한 답변: 국세공무원교육원 교육운영과장은 교육과 관련된 다양한 업무를 담당...,"교육운영과장은 교육계획의 수립, 교과시간의 운영, 강사관리, 교육생의 선발 및 학적...",0.0
6,황정연의 소득금액은 얼마인가?,[Ⅱ. 연금소득 연말정산 351 09 연금소득 원천징수영수증(연말정산용) 작성김일일...,질문이 명확하지 않습니다. 황정연이라는 이름과 관련된 구체적인 상황이나 정보를 제공...,"황정연의 소득금액은 4,700,000원입니다.",0.0
7,이태영은 어떤 공제 혜택을 받는가?,"[핵심 개정 세법 104대토보상 과세특례 적용요건 보완 (조특법 §77의2②, 조특...",죄송합니다. 이태영이라는 이름으로 특정 공제 혜택을 제공하는 세법 정보는 제가 보유...,"이태영은 국세청에서 1,500,000원의 소득공제 혜택을 받습니다.",0.0
8,봉사료가 유흥음식요금에 포함되지 않는 경우는?,"[개별소비세법 시행령 기획재정부 (환경에너지세제과) 044-215-4331, 433...",봉사료가 유흥음식요금에 포함되지 않는 경우는 다음과 같습니다:\n\n유흥음식요금에 ...,봉사료가 해당 종업원에게 지급된 사실이 확인되는 경우에는 유흥음식요금에 포함되지 않는다.,0.0
9,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은?,[143 ❚상세본 07 기타 법률수소제조용 석유가스(LPG) 부탄에 대한 환급특례 ...,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 다음과 같습니다:\n\n1....,수소를 제조하기 위한 설비에 공급되는 물품의 탄력세율은 킬로그램당 176.4원 또는...,0.0


In [23]:
# query = "2023년 4월 25일에 취업했습니다. 취업 하기 전에 지출한 월세, 신용카드 결제 금액을 연말정산시 공제 받을 수 있나요?"
# response = rag_chain.invoke(query)

In [24]:
# print(type(response))
# print(response.keys())
# print("답변:", response['llm_answer'])
# response['source_context']

<class 'dict'>
dict_keys(['source_context', 'llm_answer'])
답변: 취업 전 지출한 월세나 신용카드 결제 금액의 연말정산 공제 여부는 특정 조건에 따라 달라질 수 있습니다. 일반적으로 연말정산에서 소득공제는 해당 과세기간 내 소득이 있는 경우에만 공제 가능합니다. 따라서, 취업 이전에 소득이 없다면 해당 기간 동안의 지출에 대해서는 소득공제를 받을 수 있는 기준이 충족되지 않을 수 있습니다.

1. **월세 공제**: 월세 세액공제는 근로소득이 있는 경우 해당 과세기간에 실제로 납부한 월세에 대해 적용됩니다. 따라서, 취업 전 월세 지출은 공제 대상이 아닐 가능성이 높습니다.

2. **신용카드 공제**: 신용카드 사용금액 소득공제는 근로소득이 있는 거주자가 해당 과세연도에 사용한 금액에 대해 적용됩니다. 취업 전 사용한 금액은 소득이 없었기 때문에 공제 대상이 아닐 수 있습니다.

각 개인의 상황에 따라 다를 수 있으므로, 정확한 판단을 위해서는 세무 전문가와 상담하거나 국세청의 지침을 확인하는 것이 좋습니다. 이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.


['Ⅴ. 그 밖의 소득공제(조특법) 135 아. 유의사항 ○사업관련 경비로 처리된 종업원 명의의 신용카드 등 사용금액은 공제대상 금액에 해당되지 않으므로 원천징수의무자는 종업원의 신용카드 등 공제 신청금액에 법인(사업)경비로 처리된 금액이 포함되었는지 여부를 확인 ○법인이 ｢여신전문금융업법｣에 의한 신용카드업자로부터 종업원이 사용자로 지정된 법인신용카드를 발급받아 종업원별로 일정한도를 정하여 복리후생목적으로 사용하게 하더라도그 사용대가는 해당 종업원의 “신용카드 등 사용금액”에 포함되지 아니함 ○ 신용카드 등 사용금액 소득공제와 특별세액공제 중복 적용 여부구 분특별세액공제 항목 신용카드공제신용카드로 결제한 의료비의료비 세액공제 가능신용카드공제 가능신용카드로 결제한 보장성보험료보험료 세액공제 가능신용카드공제 불가신용카드로 결제한 학원비취학전 아동교육비 세액공제 가능* 신용카드공제 가능그 외교육비 세액공제 불가신용카드로 결제한 교복구입비교육비 세액공제 가능신용카드공제 가능신용카드로 결제한 기부금기부금 세액공제 가능신용카드공제 불가 ＊취학전 아동의 경우에는 주 1회 이상 월단위로 교습받는 학원, 체육시설 등의 수강료에 대하여 교육비 세액공제를받을 수 있습니다. 참고 ○ 현금영수증 연간 사용금액 확인 방법 - 국세청홈택스(www.hometax.go.kr) →장려금·연말정산·기부금→연말정산간소화→근로자 소득·세액공제 자료 조회→현금영수증 - 국세청홈택스(www.hometax.go.kr) →전자(세금)계산서·현금영수증·신용카드→현금영수증(근로자· 소비자) →근로자·소비자 조회/변경 ＊소득공제를 받기 위해서는 인별로 홈페이지에서 회원가입 후 현금영수증(근로자·소비자) → 소비자발급수단·전용카드 →현금영수증 발급용 휴대전화번호 및 카드관리에서 휴대전화 번호 등을 등록사례 [사례1] 총급여 68백만원인 근로자가 신용카드 등을 이용하여 올해(’24년)에는 4,300만 원(전통시장 3백만원, 대중교통 2백만원, 도서·공연 등 1백만원 포함)을 사용한 경우 올해신용카드 소득공제금액

In [ ]:
query = "2023년 4월 25일에 취업했습니다. 취업 하기 전에 지출한 월세, 신용카드 결제 금액을 연말정산시 공제 받을 수 있나요?"
response = rag_chain.invoke(query)
answer = response['llm_answer']
context_list = response['source_context']
ground_truth = "연도 중에 입사한 근로자의 연말정산시에는 근로제공기간 중에 지출한 월세와 신용카드등사용금액, 의료비,보험료, 교육비를 공제 받을 수 있습니다. 따라서 취업전인 2023년 1월 1일부터 4월 24일까지 지출한 신용카드와 의료비, 보험료는 연말정산시 공제 받을 수 없습니다."

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([query, answer])
similarity = util.cos_sim(embeddings[0], embeddings[1])
print(f"문맥 유사도: {similarity.item():.2f}")


Semantic Similarity: 0.38


In [ ]:
sample = SingleTurnSample(
    user_input=query,
    retrieved_contexts=context_list,
    response=answer,
    reference=ground_truth
)

# 평가 데이터셋 구성
eval_dataset = EvaluationDataset(samples=[sample])
eval_dataset
result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,"2023년 4월 25일에 취업했습니다. 취업 하기 전에 지출한 월세, 신용카드 결제...",[Ⅴ. 그 밖의 소득공제(조특법) 135 아. 유의사항 ○사업관련 경비로 처리된 종...,취업 전에 지출한 월세와 신용카드 결제 금액에 대한 연말정산 공제 여부는 몇 가지 ...,연도 중에 입사한 근로자의 연말정산시에는 근로제공기간 중에 지출한 월세와 신용카드등...,1.0,1.0,0.0,0.0


In [56]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'])
bleu_score = sentence_bleu([ground_truth.split()], answer.split())
rouge_scores = scorer.score(answer, ground_truth)
# semantic_similarity = cosine_similarity([ground_truth], [answer])[0][0]
print(f"BLEU점수:{bleu_score:.2f}")
print(f"Rouge1점수:{rouge_scores['rouge1']}")
print(f"RougeL점수:{rouge_scores['rougeL']}")


BLEU점수:0.00
Rouge1점수:Score(precision=0.2, recall=0.5, fmeasure=0.28571428571428575)
RougeL점수:Score(precision=0.2, recall=0.5, fmeasure=0.28571428571428575)


In [47]:
bleu_score

2.779090519026104e-155